# NBA AI - AutoML and AutoDL

PyCaret
* Main Site - https://pycaret.org/
* Docs - https://pycaret.readthedocs.io/en/latest/

AutoKeras
* Main Site - https://autokeras.com/

## Table of Contents

* [Data Setup](#data-setup)
* [AutoML Classification](#automl-classification)
* [AutoML Regression](#automl-regression)
* [AutoDL Classification](#autodl-classification)
* [AutoDL Regression](#autodl-regression)

### Imports and Global Settings

In [46]:
import datetime
import pandas as pd
from pycaret.classification import ClassificationExperiment
from pycaret.regression import RegressionExperiment
import autokeras as ak
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    accuracy_score,
    precision_score,
)

# Pandas Settings
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

Using TensorFlow backend


2023-12-30 19:18:23.864411: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 19:18:24.399192: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 19:18:24.404624: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-30 19:18:26.031924: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Load Data

In [2]:
df_2021_2022 = pd.read_csv("../data/nba_ai/cleaned_data_2021-2022.csv")

<a name="data-setup"></a>

## Data Preparation

### Train Test Split

In [3]:
def prepare_datasets(train_df, cls_target, reg_target, test_df=None, test_size=0.3):
    """
    Prepares datasets for training and testing for both classification and regression targets,
    ensuring time-sensitive splitting based on a 'date' column.

    Parameters:
    train_df (DataFrame): The training dataframe.
    cls_target (str): The name of the classification target column.
    reg_target (str): The name of the regression target column.
    test_df (DataFrame, optional): An optional testing dataframe. If not provided, a portion of the training data is used.
    test_size (float, optional): The proportion of the dataset to include in the test split (if test_df is not provided).

    Returns:
    tuple: A tuple containing six dataframes:
           (X_train, X_test, y_train_cls, y_test_cls, y_train_reg, y_test_reg).
    """

    # Sort the dataframe based on the 'date' column
    train_df = train_df.sort_values(by="date")

    # If a test dataframe is not provided, split the training dataframe
    if test_df is None:
        X_train, X_test, y_train, y_test = train_test_split(
            train_df.drop([cls_target, reg_target], axis=1),
            train_df[[cls_target, reg_target]],
            test_size=test_size,
            shuffle=False,  # Important to maintain time order
        )
    else:
        # If a test dataframe is provided, ensure it is also sorted by date
        test_df = test_df.sort_values(by="date")

        # Use provided test dataframe and separate features and targets
        X_train = train_df.drop([cls_target, reg_target], axis=1)
        y_train = train_df[[cls_target, reg_target]]
        X_test = test_df.drop([cls_target, reg_target], axis=1)
        y_test = test_df[[cls_target, reg_target]]

    # Separate classification and regression targets
    y_train_cls = y_train[[cls_target]]
    y_train_reg = y_train[[reg_target]]
    y_test_cls = y_test[[cls_target]]
    y_test_reg = y_test[[reg_target]]

    return X_train, X_test, y_train_cls, y_test_cls, y_train_reg, y_test_reg

In [4]:
X_train, X_test, y_train_cls, y_test_cls, y_train_reg, y_test_reg = prepare_datasets(
    df_2021_2022, "CLS_TARGET", "REG_TARGET", test_df=None
)

### Features

In [5]:
betting_feature_set = [
    "home_opening_spread",
    "road_opening_spread",
    "opening_total",
    "home_closing_spread",
    "road_closing_spread",
    "closing_total",
    "home_moneyline",
    "road_moneyline",
]

base_feature_set = [
    "day_of_season",
    "home_win_pct",
    "road_win_pct",
    "home_win_pct_l2w",
    "road_win_pct_l2w",
    "home_avg_pts",
    "road_avg_pts",
    "home_avg_pts_l2w",
    "road_avg_pts_l2w",
    "home_avg_oeff",
    "road_avg_oeff",
    "home_avg_oeff_l2w",
    "road_avg_oeff_l2w",
    "home_avg_deff",
    "road_avg_deff",
    "home_avg_deff_l2w",
    "road_avg_deff_l2w",
    "home_avg_eFG%",
    "road_avg_eFG%",
    "home_avg_eFG%_l2w",
    "road_avg_eFG%_l2w",
    "home_avg_TOV%",
    "road_avg_TOV%",
    "home_avg_TOV%_l2w",
    "road_avg_TOV%_l2w",
    "home_avg_ORB%",
    "road_avg_ORB%",
    "home_avg_ORB%_l2w",
    "road_avg_ORB%_l2w",
    "home_avg_FT%",
    "road_avg_FT%",
    "home_avg_FT%_l2w",
    "road_avg_FT%_l2w",
    "home_avg_pts_allowed",
    "road_avg_pts_allowed",
    "home_avg_pts_allowed_l2w",
    "road_avg_pts_allowed_l2w",
]

features_to_prepare = [
    "home_team",
    "road_team",
    "home_team_rest_days",
    "road_team_rest_days",
    "home_team_starting_lineup",
    "road_team_starting_lineup",
]

In [6]:
X_train = X_train[base_feature_set]
X_test = X_test[base_feature_set]

### Combined Data

In [7]:
combined_train_df = pd.concat([X_train, y_train_cls, y_train_reg], axis=1)
combined_test_df = pd.concat([X_test, y_test_cls, y_test_reg], axis=1)

In [8]:
combined_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 926 entries, 0 to 925
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   day_of_season             926 non-null    int64  
 1   home_win_pct              926 non-null    float64
 2   road_win_pct              926 non-null    float64
 3   home_win_pct_l2w          926 non-null    float64
 4   road_win_pct_l2w          926 non-null    float64
 5   home_avg_pts              926 non-null    float64
 6   road_avg_pts              926 non-null    float64
 7   home_avg_pts_l2w          926 non-null    float64
 8   road_avg_pts_l2w          926 non-null    float64
 9   home_avg_oeff             926 non-null    float64
 10  road_avg_oeff             926 non-null    float64
 11  home_avg_oeff_l2w         926 non-null    float64
 12  road_avg_oeff_l2w         926 non-null    float64
 13  home_avg_deff             926 non-null    float64
 14  road_avg_d

In [9]:
combined_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 926 to 1322
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   day_of_season             397 non-null    int64  
 1   home_win_pct              397 non-null    float64
 2   road_win_pct              397 non-null    float64
 3   home_win_pct_l2w          397 non-null    float64
 4   road_win_pct_l2w          397 non-null    float64
 5   home_avg_pts              397 non-null    float64
 6   road_avg_pts              397 non-null    float64
 7   home_avg_pts_l2w          397 non-null    float64
 8   road_avg_pts_l2w          397 non-null    float64
 9   home_avg_oeff             397 non-null    float64
 10  road_avg_oeff             397 non-null    float64
 11  home_avg_oeff_l2w         397 non-null    float64
 12  road_avg_oeff_l2w         397 non-null    float64
 13  home_avg_deff             397 non-null    float64
 14  road_av

<a name="automl-classification"></a>

## AutoML Classification

### Setup and Preprocessing

In [10]:
py_cls = ClassificationExperiment()

In [11]:
setup_params_cls = {
    "data": combined_train_df,
    "test_data": combined_test_df,
    "target": "CLS_TARGET",
    "ignore_features": ["REG_TARGET"],
}

In [12]:
py_cls.setup(**setup_params_cls)

,Description,Value
0,Session id,7882
1,Target,CLS_TARGET
2,Target type,Binary
3,Original data shape,"(1323, 39)"
4,Transformed data shape,"(1323, 38)"
5,Transformed train set shape,"(926, 38)"
6,Transformed test set shape,"(397, 38)"
7,Ignore features,1
8,Numeric features,37
9,Preprocess,True


### Compare Models

In [13]:
best_model_cls = py_cls.compare_models(
    turbo=True, sort="Accuracy", exclude=["catboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.5412,0.0000,0.4024,0.5472,0.4406,0.0668,0.0797,0.0520
dummy,Dummy Classifier,0.5313,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0580
lr,Logistic Regression,0.5292,0.5474,0.3679,0.5261,0.4086,0.0398,0.0505,0.5820
nb,Naive Bayes,0.5281,0.5459,0.0341,0.0455,0.0390,-0.0027,-0.0028,0.0570
lda,Linear Discriminant Analysis,0.5271,0.5531,0.4190,0.5105,0.4435,0.0421,0.0480,0.0470
qda,Quadratic Discriminant Analysis,0.5097,0.4927,0.4405,0.4693,0.4511,0.0112,0.0100,0.0520
svm,SVM - Linear Kernel,0.5054,0.0000,0.5511,0.3862,0.3778,0.0180,0.0314,0.0480
rf,Random Forest Classifier,0.4978,0.4778,0.3589,0.4076,0.3794,-0.0212,-0.0222,0.3980
dt,Decision Tree Classifier,0.4913,0.4867,0.4141,0.4078,0.4097,-0.0266,-0.0266,0.0630
et,Extra Trees Classifier,0.4913,0.4924,0.3548,0.4708,0.3828,-0.0329,-0.0271,0.3040


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [14]:
print(best_model_cls)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=7882, solver='auto',
                tol=0.0001)


### Create Selected Model

In [15]:
model_cls = py_cls.create_model("ridge")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5484,0.0000,0.3636,0.5333,0.4324,0.0792,0.0832
1,0.5054,0.0000,0.4545,0.4762,0.4651,0.0056,0.0056
2,0.5484,0.0000,0.6364,0.5185,0.5714,0.1045,0.1070
3,0.4731,0.0000,0.5000,0.4490,0.4731,-0.0507,-0.0510
4,0.5161,0.0000,0.4651,0.4762,0.4706,0.0252,0.0252
5,0.5269,0.0000,0.3953,0.4857,0.4359,0.0358,0.0364
6,0.5761,0.0000,0.3721,0.5714,0.4507,0.1300,0.1379
7,0.5217,0.0000,0.2791,0.4800,0.3529,0.0141,0.0154
8,0.5978,0.0000,0.3953,0.6071,0.4789,0.1746,0.1853


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

### Tune Selected Model

In [16]:
tuned_model_cls = py_cls.tune_model(model_cls)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5914,0.0000,0.4545,0.5882,0.5128,0.1708,0.1750
1,0.5161,0.0000,0.4545,0.4878,0.4706,0.0261,0.0261
2,0.5484,0.0000,0.6364,0.5185,0.5714,0.1045,0.1070
3,0.4839,0.0000,0.5000,0.4583,0.4783,-0.0305,-0.0306
4,0.5161,0.0000,0.4651,0.4762,0.4706,0.0252,0.0252
5,0.5269,0.0000,0.3953,0.4857,0.4359,0.0358,0.0364
6,0.5761,0.0000,0.3721,0.5714,0.4507,0.1300,0.1379
7,0.5109,0.0000,0.2558,0.4583,0.3284,-0.0098,-0.0108
8,0.5978,0.0000,0.3953,0.6071,0.4789,0.1746,0.1853


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [17]:
print(tuned_model_cls)

RidgeClassifier(alpha=1.13, class_weight=None, copy_X=True, fit_intercept=False,
                max_iter=None, positive=False, random_state=7882, solver='auto',
                tol=0.0001)


### Evaluate Model

In [18]:
py_cls.evaluate_model(tuned_model_cls)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [19]:
# py_cls.interpret_model(tuned_model_cls)

In [20]:
train_predictions_cls = py_cls.predict_model(tuned_model_cls, data=X_train)

In [21]:
test_predictions_cls = py_cls.predict_model(tuned_model_cls, data=X_test)

In [22]:
train_accuracy = accuracy_score(train_predictions_cls["prediction_label"], y_train_cls)
train_precision = precision_score(
    train_predictions_cls["prediction_label"], y_train_cls
)

test_accuracy = accuracy_score(test_predictions_cls["prediction_label"], y_test_cls)
test_precision = precision_score(test_predictions_cls["prediction_label"], y_test_cls)

In [23]:
print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Train Precision: {train_precision:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Precision: {test_precision:.2f}")

Train Accuracy: 0.58
Train Precision: 0.44
Test Accuracy: 0.49
Test Precision: 0.10


### Model Finalization and Storage

In [24]:
final_model_cls = py_cls.finalize_model(tuned_model_cls)

In [25]:
problem_type = "Classification"
base_model = "Ridge"
train_performance = round(train_accuracy * 100)
test_performance = round(test_accuracy * 100)

model_id = f"{problem_type}_{base_model}_{train_performance}_{test_performance}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

model_id

'Classification_Ridge_58_49_2023-12-30_19-05-27'

In [26]:
# py_cls.save_model(final_model_cls, f"../models/AutoML/{model_id}")

<a name="automl-regression"></a>

## AutoML Regression

### Setup and Preprocessing

In [27]:
py_reg = RegressionExperiment()

In [28]:
setup_params_reg = {
    "data": combined_train_df,
    "test_data": combined_test_df,
    "target": "REG_TARGET",
    "ignore_features": ["CLS_TARGET"],
}

In [29]:
py_reg.setup(**setup_params_reg)

,Description,Value
0,Session id,1779
1,Target,REG_TARGET
2,Target type,Regression
3,Original data shape,"(1323, 39)"
4,Transformed data shape,"(1323, 38)"
5,Transformed train set shape,"(926, 38)"
6,Transformed test set shape,"(397, 38)"
7,Ignore features,1
8,Numeric features,37
9,Preprocess,True


### Compare Models

In [30]:
best_model_reg = py_reg.compare_models(turbo=True, sort="MAE", exclude=["catboost"])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,11.5953,215.7389,14.6203,0.0261,1.3017,1.2040,0.0390
br,Bayesian Ridge,11.6106,214.8984,14.6018,0.0307,1.4340,1.1316,0.0420
en,Elastic Net,11.6150,215.2043,14.6079,0.0297,1.3771,1.1641,0.0500
lasso,Lasso Regression,11.6336,215.8861,14.6318,0.0264,1.3962,1.1608,0.0470
llar,Lasso Least Angle Regression,11.6338,215.8878,14.6319,0.0264,1.3962,1.1608,0.0410
lr,Linear Regression,11.6810,220.0833,14.7612,0.0054,1.2484,1.2623,0.0520
huber,Huber Regressor,11.7142,221.2243,14.7802,0.0029,1.3663,1.1825,0.0650
et,Extra Trees Regressor,11.7179,216.2056,14.6578,0.0259,1.2562,1.2544,0.3980
ada,AdaBoost Regressor,11.7716,217.7181,14.7173,0.0143,1.1651,1.3180,0.2270
rf,Random Forest Regressor,11.8601,221.6942,14.8410,0.0012,1.2368,1.2408,0.7880


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [31]:
print(best_model_reg)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, positive=False,
      random_state=1779, solver='auto', tol=0.0001)


### Create Selected Model

In [32]:
model_reg = py_reg.create_model("ridge")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13.7807,307.3384,17.5311,-0.3950,1.2256,1.6979
1,11.0267,187.3829,13.6888,-0.0155,1.3058,1.1949
2,11.8180,208.6400,14.4444,0.0550,1.3157,1.0732
3,12.2200,264.3194,16.2579,-0.0493,1.2371,1.4120
4,11.7251,203.6416,14.2703,0.0800,1.2917,1.0816
5,10.3634,171.7124,13.1039,0.1198,1.1946,1.0789
6,12.3453,237.1913,15.4010,0.0729,1.4645,0.9499
7,9.6131,155.8291,12.4832,0.1217,1.2033,1.1248
8,11.8150,201.8327,14.2068,0.1637,1.4881,1.1739


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

### Tune Selected Model

In [33]:
tuned_model_reg = py_reg.tune_model(model_reg)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,13.8383,305.1249,17.4678,-0.3849,1.2730,1.6725
1,10.8200,180.8384,13.4476,0.0200,1.2721,1.1667
2,11.8488,210.3735,14.5043,0.0472,1.3511,1.0774
3,12.0520,260.9180,16.1530,-0.0358,1.2175,1.3572
4,11.7222,202.1101,14.2165,0.0869,1.3293,1.0721
5,10.2145,167.8474,12.9556,0.1397,1.2681,1.0359
6,12.4131,238.9981,15.4596,0.0659,1.4486,0.9461
7,9.4611,152.8546,12.3634,0.1385,1.1883,1.0794
8,11.8306,202.6174,14.2344,0.1604,1.4839,1.1222


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [34]:
print(tuned_model_reg)

Ridge(alpha=8.9, copy_X=True, fit_intercept=True, max_iter=None, positive=False,
      random_state=1779, solver='auto', tol=0.0001)


### Evaluate Model

In [35]:
py_reg.evaluate_model(tuned_model_reg)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [36]:
# py_reg.interpret_model(tuned_model_reg)

In [37]:
train_predictions_reg = py_reg.predict_model(tuned_model_reg, data=X_train)

In [38]:
test_predictions_reg = py_reg.predict_model(tuned_model_reg, data=X_test)

In [39]:
train_mae = mean_absolute_error(train_predictions_reg["prediction_label"], y_train_reg)
train_r2 = r2_score(train_predictions_reg["prediction_label"], y_train_reg)

test_mae = mean_absolute_error(test_predictions_reg["prediction_label"], y_test_reg)
test_r2 = r2_score(test_predictions_reg["prediction_label"], y_test_reg)

In [40]:
print(f"Train MAE: {train_mae:.2f}")
print(f"Train R2: {train_r2:.2f}")
print(f"Test MAE: {test_mae:.2f}")
print(f"Test R2: {test_r2:.2f}")

Train MAE: 11.16
Train R2: -7.87
Test MAE: 12.54
Test R2: -15.77


### Model Finalization and Storage

In [43]:
final_model_reg = py_reg.finalize_model(tuned_model_reg)

In [44]:
problem_type = "Regression"
base_model = "Ridge"
train_performance = round(train_mae, 2)
test_performance = round(test_mae, 2)

model_id = f"{problem_type}_{base_model}_{train_performance}_{test_performance}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

model_id

'Regression_Ridge_11.16_12.54_2023-12-30_19-14-58'

In [45]:
py_reg.save_model(final_model_reg, f"../models/AutoML/{model_id}")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['day_of_season', 'home_win_pct',
                                              'road_win_pct', 'home_win_pct_l2w',
                                              'road_win_pct_l2w', 'home_avg_pts',
                                              'road_avg_pts', 'home_avg_pts_l2w',
                                              'road_avg_pts_l2w',
                                              'home_avg_oeff', 'road_avg_oeff',
                                              'home_avg_oeff_l2w',
                                              'road_avg_oeff_l2w',
                                              'home_av...
                                                               missing_values=nan,
                                                               strategy='most_frequent',
                        

<a name="autodl-classification"></a>

## AutoDL Classification

In [47]:
ak_cls = ak.StructuredDataClassifier(
    max_trials=10,
    overwrite=True,
    loss="accuracy",
)

In [48]:
ak_cls.fit(X_train, y_train_cls)

Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.42405062913894653

Best val_accuracy So Far: 0.6012658476829529
Total elapsed time: 00h 01m 21s
29/29 [==============================] - 2s 5ms/step - loss: 0.8193 - accuracy: 0.4914
INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [49]:
# Evaluate the best model with testing data.
print(ak_cls.evaluate(X_test, y_test_cls))

13/13 [==============================] - 0s 4ms/step - loss: 0.7759 - accuracy: 0.4937
[0.7758802771568298, 0.49370276927948]


In [51]:
ak_cls_model = ak_cls.export_model()
ak_cls_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 37)]              0         
                                                                 
 multi_category_encoding (M  (None, 37)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 37)                75        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                1216      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dropout (Dropout)           (None, 32)                0     

<a id=evaluate></a>

### Evaluate Model

In [52]:
train_pred = ak_cls_model.predict(X_train)
test_pred = ak_cls_model.predict(X_test)

13/13 [==============================] - 0s 2ms/step


In [53]:
train_pred = train_pred.flatten()
train_pred_labels = [True if x > 0.5 else False for x in train_pred]

In [54]:
test_pred = test_pred.flatten()
test_pred_labels = [True if x > 0.5 else False for x in test_pred]

In [58]:
train_accuracy = accuracy_score(y_train_cls, train_pred_labels)
train_precision = precision_score(y_train_cls, train_pred_labels)
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Training Precision: {train_precision:.2f}")

Training Accuracy: 0.47
Training Precision: 0.46


In [60]:
test_accuracy = accuracy_score(y_test_cls, test_pred_labels)
test_auc = precision_score(y_test_cls, test_pred_labels)
print(f"Testing Accuracy: {test_accuracy:.2f}")
print(f"Testing Precision: {test_precision:.2f}")

Testing Accuracy: 0.49
Testing Precision: 0.10


### Model Storage

In [62]:
problem_type = "Classification"
base_model = "AutoKeras"
train_performance = round(train_accuracy * 100)
test_performance = round(test_accuracy * 100)

model_id = f"{problem_type}_{base_model}_{train_performance}_{test_performance}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

model_id

'Classification_AutoKeras_47_49_2023-12-30_19-32-24'

In [63]:
ak_cls_model.save(f"../models/AutoDL/{model_id}", save_format="tf")

INFO:tensorflow:Assets written to: ../models/AutoDL/Classification_AutoKeras_47_49_2023-12-30_19-32-24/assets


INFO:tensorflow:Assets written to: ../models/AutoDL/Classification_AutoKeras_47_49_2023-12-30_19-32-24/assets


<a name="autodl-regression"></a>

## AutoDL Regression

In [65]:
ak_reg = ak.StructuredDataRegressor(
    max_trials=10,
    overwrite=True,
    loss="mae",
)

In [66]:
ak_reg.fit(X_train, y_train_reg)

Trial 10 Complete [00h 00m 08s]
val_loss: 12.461597442626953

Best val_loss So Far: 11.76154613494873
Total elapsed time: 00h 01m 20s
Epoch 1/18
29/29 [==============================] - 1s 4ms/step - loss: 12.0720 - mean_squared_error: 225.8528
Epoch 2/18
29/29 [==============================] - 0s 4ms/step - loss: 11.9769 - mean_squared_error: 222.0988
Epoch 3/18
29/29 [==============================] - 0s 3ms/step - loss: 11.8612 - mean_squared_error: 218.5507
Epoch 4/18
29/29 [==============================] - 0s 3ms/step - loss: 11.6540 - mean_squared_error: 212.5936
Epoch 5/18
29/29 [==============================] - 0s 4ms/step - loss: 11.3659 - mean_squared_error: 205.1582
Epoch 6/18
29/29 [==============================] - 0s 4ms/step - loss: 11.0927 - mean_squared_error: 199.3703
Epoch 7/18
29/29 [==============================] - 0s 5ms/step - loss: 10.9553 - mean_squared_error: 198.8292
Epoch 8/18
29/29 [==============================] - 0s 6ms/step - loss: 10.9205 - mean_sq

INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [67]:
print(ak_reg.evaluate(X_test, y_test_reg))

13/13 [==============================] - 0s 3ms/step - loss: 12.0684 - mean_squared_error: 240.2173
[12.068380355834961, 240.21730041503906]


In [68]:
ak_reg_model = ak_reg.export_model()
ak_reg_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 37)]              0         
                                                                 
 multi_category_encoding (M  (None, 37)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 37)                75        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                1216      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

<a id=evaluate></a>

### Evaluate Model

In [69]:
train_pred = ak_reg_model.predict(X_train)
test_pred = ak_reg_model.predict(X_test)

13/13 [==============================] - 0s 2ms/step


In [72]:
train_mae = mean_absolute_error(y_train_reg, train_pred)
train_r2 = r2_score(y_train_reg, train_pred)
print(f"Training MAE: {train_mae:.2f}")
print(f"Training R2: {train_r2:.2f}")

Training MAE: 10.24
Training R2: 0.18


In [73]:
test_mae = mean_absolute_error(y_test_reg, test_pred)
test_r2 = r2_score(y_test_reg, test_pred)
print(f"Testing MAE: {test_mae:.2f}")
print(f"Testing R2: {test_r2:.2f}")

Testing MAE: 12.07
Testing R2: 0.07


### Model Storage

In [74]:
problem_type = "Regression"
base_model = "AutoKeras"
train_performance = round(train_mae, 2)
test_performance = round(test_mae, 2)

model_id = f"{problem_type}_{base_model}_{train_performance}_{test_performance}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

model_id

'Regression_AutoKeras_10.24_12.07_2023-12-30_19-40-49'

In [75]:
ak_reg_model.save(f"../models/AutoDL/{model_id}", save_format="tf")

INFO:tensorflow:Assets written to: ../models/AutoDL/Regression_AutoKeras_10.24_12.07_2023-12-30_19-40-49/assets


INFO:tensorflow:Assets written to: ../models/AutoDL/Regression_AutoKeras_10.24_12.07_2023-12-30_19-40-49/assets
